In [1]:
###!wget http://courses.cs.washington.edu/courses/cse341/07wi/handouts/hamlet.txt
###!ls

In [2]:
import pyspark
sc = pyspark.SparkContext('local[*]')

In [3]:
# do something to prove it works
rdd = sc.parallelize(range(1000))
###rdd.takeSample(False, 5)

In [36]:
%%time
raw_hamlet_zipped = sc.textFile('hamlet.txt').zipWithIndex()
raw_hamlet = raw_hamlet_zipped.map(
    lambda line: 'hamlet@{}\t\t{}'.format(line[1],line[0]) if len(line[0])>0 else 'hamlet@{}'.format(line[1]))
raw_hamlet.take(25)

CPU times: user 23.4 ms, sys: 3.13 ms, total: 26.5 ms
Wall time: 393 ms


In [25]:
def filter_hamlet_speaks(line):
    if "Ham." in line: return True
    return False
hamlet_spoken_lines = raw_hamlet.filter(lambda line: filter_hamlet_speaks(line))
split_hamlet = hamlet_spoken_lines.map(lambda line: line.split('\t'))
split_hamlet.take(25)

[['hamlet@336',
  '',
  '  Ham. [aside] A little more than kin, and less than kind!'],
 ['hamlet@338', '', "  Ham. Not so, my lord. I am too much i' th' sun."],
 ['hamlet@345', '', '  Ham. Ay, madam, it is common.'],
 ['hamlet@348', '', "  Ham. Seems, madam, Nay, it is. I know not 'seems.'"],
 ['hamlet@392', '', '  Ham. I shall in all my best obey you, madam.'],
 ['hamlet@402', '', '  Ham. O that this too too solid flesh would melt,'],
 ['hamlet@437', '', '  Ham. I am glad to see you well.'],
 ['hamlet@440',
  '',
  "  Ham. Sir, my good friend- I'll change that name with you."],
 ['hamlet@444',
  '',
  '  Ham. I am very glad to see you.- [To Bernardo] Good even, sir.-  '],
 ['hamlet@447', '', '  Ham. I would not hear your enemy say so,'],
 ['hamlet@454', '', '  Ham. I prithee do not mock me, fellow student.'],
 ['hamlet@457', '', "  Ham. Thrift, thrift, Horatio! The funeral bak'd meats"],
 ['hamlet@463', '', "  Ham. In my mind's eye, Horatio."],
 ['hamlet@465', '', '  Ham. He was a man

In [28]:
spoken_count = 0
spoken_101 = list()
spoken_count = hamlet_spoken_lines.count()
print (spoken_count)
#spoken_collect = hamlet_spoken_lines.collect()
#spoken_101 = spoken_collect[100]
spoken_101 = hamlet_spoken_lines.collect()[100]
print (spoken_101)

358
hamlet@1521		  Ham. I will tell you why. So shall my anticipation prevent your


In [33]:
def format_id(x):
    id = x[0].split('@')[1]
    results = list()
    results.append(id)
    if len(x) > 1:
        for y in x[1:]:
            results.append('{}{}'.format(y,'|'))
    results.append('|')
    return results

hamlet_with_ids = split_hamlet.map(lambda line: format_id(line))
hamlet_with_ids.take(25)

[['336',
  '|',
  '  Ham. [aside] A little more than kin, and less than kind!|',
  '|'],
 ['338', '|', "  Ham. Not so, my lord. I am too much i' th' sun.|", '|'],
 ['345', '|', '  Ham. Ay, madam, it is common.|', '|'],
 ['348', '|', "  Ham. Seems, madam, Nay, it is. I know not 'seems.'|", '|'],
 ['392', '|', '  Ham. I shall in all my best obey you, madam.|', '|'],
 ['402', '|', '  Ham. O that this too too solid flesh would melt,|', '|'],
 ['437', '|', '  Ham. I am glad to see you well.|', '|'],
 ['440',
  '|',
  "  Ham. Sir, my good friend- I'll change that name with you.|",
  '|'],
 ['444',
  '|',
  '  Ham. I am very glad to see you.- [To Bernardo] Good even, sir.-  |',
  '|'],
 ['447', '|', '  Ham. I would not hear your enemy say so,|', '|'],
 ['454', '|', '  Ham. I prithee do not mock me, fellow student.|', '|'],
 ['457', '|', "  Ham. Thrift, thrift, Horatio! The funeral bak'd meats|", '|'],
 ['463', '|', "  Ham. In my mind's eye, Horatio.|", '|'],
 ['465', '|', '  Ham. He was a man

In [34]:
real_text = hamlet_with_ids.filter(lambda line: len(line) > 1)
hamlet_text_only = real_text.map(lambda line: [l for l in line if l != ''])
hamlet_text_only.take(25)

[['336',
  '|',
  '  Ham. [aside] A little more than kin, and less than kind!|',
  '|'],
 ['338', '|', "  Ham. Not so, my lord. I am too much i' th' sun.|", '|'],
 ['345', '|', '  Ham. Ay, madam, it is common.|', '|'],
 ['348', '|', "  Ham. Seems, madam, Nay, it is. I know not 'seems.'|", '|'],
 ['392', '|', '  Ham. I shall in all my best obey you, madam.|', '|'],
 ['402', '|', '  Ham. O that this too too solid flesh would melt,|', '|'],
 ['437', '|', '  Ham. I am glad to see you well.|', '|'],
 ['440',
  '|',
  "  Ham. Sir, my good friend- I'll change that name with you.|",
  '|'],
 ['444',
  '|',
  '  Ham. I am very glad to see you.- [To Bernardo] Good even, sir.-  |',
  '|'],
 ['447', '|', '  Ham. I would not hear your enemy say so,|', '|'],
 ['454', '|', '  Ham. I prithee do not mock me, fellow student.|', '|'],
 ['457', '|', "  Ham. Thrift, thrift, Horatio! The funeral bak'd meats|", '|'],
 ['463', '|', "  Ham. In my mind's eye, Horatio.|", '|'],
 ['465', '|', '  Ham. He was a man

In [35]:
clean_hamlet = real_text.map(lambda line: [l.replace('|',' ') for l in line if l != '|'])
clean_hamlet.take(25)

[['336', '  Ham. [aside] A little more than kin, and less than kind! '],
 ['338', "  Ham. Not so, my lord. I am too much i' th' sun. "],
 ['345', '  Ham. Ay, madam, it is common. '],
 ['348', "  Ham. Seems, madam, Nay, it is. I know not 'seems.' "],
 ['392', '  Ham. I shall in all my best obey you, madam. '],
 ['402', '  Ham. O that this too too solid flesh would melt, '],
 ['437', '  Ham. I am glad to see you well. '],
 ['440', "  Ham. Sir, my good friend- I'll change that name with you. "],
 ['444',
  '  Ham. I am very glad to see you.- [To Bernardo] Good even, sir.-   '],
 ['447', '  Ham. I would not hear your enemy say so, '],
 ['454', '  Ham. I prithee do not mock me, fellow student. '],
 ['457', "  Ham. Thrift, thrift, Horatio! The funeral bak'd meats "],
 ['463', "  Ham. In my mind's eye, Horatio. "],
 ['465', '  Ham. He was a man, take him for all in all. '],
 ['468', '  Ham. Saw? who? '],
 ['470', '  Ham. The King my father? '],
 ['475', "  Ham. For God's love let me hear! "],